In [1]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt
!pip install roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 15162, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 15162 (delta 7), reused 9 (delta 2), pack-reused 15138
Receiving objects: 100% (15162/15162), 14.15 MiB | 781.00 KiB/s, done.
Resolving deltas: 100% (10391/10391), done.


In [92]:
import torch

from roboflow import Roboflow
rf = Roboflow(api_key="RtdzCZNW8mDA4ilAU416")
project = rf.workspace("yolov7-twb3s").project("step2-v1_2-mud-illu-6-beixinyao-vwcvx")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Step2-v1_2-mud-illu-6-beixinyao-2 in yolov5pytorch:: 100%|████████████████████████████████████████| 5270/5270 [00:11<00:00, 477.66it/s]


In [1]:
import numpy as np
import os

def convert_to_bounding_box(segmentation_label):
    x_coord, y_coord = [], []
    for i in range(0, len(segmentation_label), 2):
        x_coord.append(segmentation_label[i])
        y_coordinates.append(segmentation_label[i+1])
    return convert_to_yolov5(min(x_coord), min(y_coord), max(x_coord), max(y_coord))

def convert_to_yolov5(xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    return np.array((x_center, y_center, width, height))


def from_xmin_ymin_xmax_ymax_to_yolov5(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
            for i, line in enumerate(lines):
                values = line.split()
                class_label = values[0]
                polygon = [float(val) for val in values[1:]]
                bbox = convert_to_bounding_box(polygon)
                lines[i] = f"{class_label} {' '.join(map(str, bbox))}\n"
            with open(file_path, 'w') as file:
                file.writelines(lines)
                
#from_xmin_ymin_xmax_ymax_to_yolov5('Step2-v1_2-mud-illu-6-beixinyao-2/train/labels')
#from_xmin_ymin_xmax_ymax_to_yolov5('Step2-v1_2-mud-illu-6-beixinyao-2/valid/labels')
#from_xmin_ymin_xmax_ymax_to_yolov5('Step2-v1_2-mud-illu-6-beixinyao-2/test/labels')




In [2]:
import numpy as np

def iou(box1, box2):
    """
    Calculates the Intersection over Union (IoU) of two bounding boxes.
    box1 and box2 are arrays of length 4: [x_center, y_center, width, height].
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate intersection area
    x_left = max(x1 - w1/2, x2 - w2/2)
    y_top = max(y1 - h1/2, y2 - h2/2)
    x_right = min(x1 + w1/2, x2 + w2/2)
    y_bottom = min(y1 + h1/2, y2 + h2/2)
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    # Calculate union area
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou


def match_boxes(boxes1, boxes2):
    """
    Matches boxes in boxes1 to boxes in boxes2 based on IoU.
    boxes1 and boxes2 are arrays of shape (n_boxes, 5) where the first
    element of each row is the class (0 or 1) and the remaining 4 elements
    are [x_center, y_center, width, height].
    Returns a list of tuples, where each tuple contains the indices of the
    matched boxes in boxes1 and boxes2, or None if no match was found.
    """
    n_boxes1 = boxes1.shape[0]
    n_boxes2 = boxes2.shape[0]
    
    # Calculate IoU matrix
    iou_matrix = np.zeros((n_boxes1, n_boxes2))
    for i in range(n_boxes1):
        for j in range(n_boxes2):
            iou_matrix[i,j] = iou(boxes1[i], boxes2[j])
    
    # Find best matches
    matches = []
    
    for i in range(n_boxes1):
        j = np.argmax(iou_matrix[i])
        if iou_matrix[i,j] > 0:
            # Check if box j is already matched to another box
            matched_boxes = [m[1] for m in matches if m is not None]
            if j not in matched_boxes:
                matches.append((i, j))
            else:
                # If box j is already matched, find the best unmatched box
                ious = iou_matrix[i,:]
                ious[matched_boxes] = 0
                j2 = np.argmax(ious)
                if iou_matrix[i,j2] >= 0.5:
                    matches.append((i, j2))
    
    return matches


In [3]:
def convert_to_bounding_box(segmentation_label):
    x_coordinates = []
    y_coordinates = []
    for i in range(0, len(segmentation_label), 2):
        x_coordinates.append(segmentation_label[i])
        y_coordinates.append(segmentation_label[i+1])
    xmin = min(x_coordinates)
    xmax = max(x_coordinates)
    ymin = min(y_coordinates)
    ymax = max(y_coordinates)
    return convert_to_yolov5(xmin, ymin, xmax, ymax)

def convert_to_yolov5(xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    return [x_center, y_center, width, height]


In [ ]:
# Draw the polygon on the image

In [2]:
import numpy as np
import cv2

# Load the image
img = cv2.imread('Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0006_JPG_jpg.rf.a4a0254a88a5d34201a8628a1ff1ab93.jpg')
#vertices = np.array([[0.503125, 0.75], [0.5, 0.753125], [0.5, 0.76875], [0.503125, 0.771875], [0.503125, 0.775], [0.509375, 0.78125], [0.509375, 0.784375], [0.51875, 0.79375], [0.521875, 0.79375], [0.534375, 0.80625], [0.5375, 0.80625], [0.540625, 0.809375], [0.59375, 0.809375],  [0.596875, 0.80625], [0.603125, 0.80625], [0.60625, 0.803125], [0.60625, 0.796875], [0.609375, 0.79375], [0.609375, 0.778125], [0.6125, 0.775], [0.6125, 0.76875], [0.621875, 0.759375], [0.621875, 0.753125], [0.61875, 0.753125], [0.615625, 0.75], [0.590625, 0.75], [0.5875, 0.753125], [0.571875, 0.753125], [0.56875, 0.75625], [0.54375, 0.75625], [0.540625, 0.753125], [0.53125, 0.753125], [0.528125, 0.75]])
# Scale the vertices to the image size
height, width, _ = img.shape
vertices[:, 0] *= width
vertices[:, 1] *= height

# Draw the polygon on the image
cv2.polylines(img, [vertices.astype(np.int32)], True, (0, 255, 0), thickness=2)

# Show the image with the segmentation
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'vertices' is not defined

In [ ]:
### computes metrics on OD model

In [62]:
!python yolov5/val.py --weights OD/trained_on_bbox/train/bestOD.pt --img 320 --data Step2-v1_2-mud-illu-6-beixinyao-2/data.yaml

val: data=Step2-v1_2-mud-illu-6-beixinyao-2/data.yaml, weights=['OD/trained_on_bbox/train/bestOD.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-105-g226a5e4 Python-3.9.13 torch-1.13.1+cu117 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/vali
val: New cache created: /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/valid/labels.cache
                 Class     Images  Instances          P          R      mAP50   
                   all         51        939      0.977      0.947      0.981      0.773
                  coal         51        308      0.986      0.977      0.994      0.806
     

In [ ]:
 0.977      0.947      0.981      0.773

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import numpy as np
from sklearn.metrics import pairwise_distances

def compute_recall(pred_boxes, pred_classes, pred_confidence, gt_boxes, gt_classes, iou_threshold=0.5, confidence_threshold=0.5):
    # Filter predicted boxes based on confidence score
    positive_mask = pred_confidence >= confidence_threshold
    pred_boxes = pred_boxes[positive_mask]
    pred_classes = pred_classes[positive_mask]

    # Compute IoU between predicted boxes and ground truth boxes
    iou_matrix = pairwise_distances(pred_boxes, gt_boxes, metric=iou)
    matches = iou_matrix > iou_threshold

    # Count true positives for each ground truth box
    tp_counts = np.sum(matches, axis=0)
    tp_mask = tp_counts > 0

    # Compute recall
    recall = np.sum(tp_mask) / gt_boxes.shape[0]

    return recall

def compute_precision(pred_boxes, pred_classes, pred_confidence, gt_boxes, gt_classes, iou_threshold=0.5, confidence_threshold=0.5):
    # Filter predicted boxes based on confidence score
    positive_mask = pred_confidence >= confidence_threshold
    pred_boxes = pred_boxes[positive_mask]
    pred_classes = pred_classes[positive_mask]

    # Compute IoU between predicted boxes and ground truth boxes
    iou_matrix = pairwise_distances(pred_boxes, gt_boxes, metric=iou)
    matches = iou_matrix > iou_threshold

    # Count true positives for each predicted box
    tp_counts = np.sum(matches, axis=1)
    tp_mask = tp_counts > 0

    # Compute precision
    precision = np.sum(tp_mask) / pred_boxes.shape[0]

    return precision


In [ ]:
# this loads the predictions made by a segment model from EXP after have run !python yolov5/segment/predict.py --save-txt --weights seg_model.pt --img 320  --source 'Step2-v1_2-mud-illu-6-beixinyao-2/test/images/*.jpg'
# and converts them to bbox and then creates the dict for metrics

In [5]:
import os
import torch
import numpy as np
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.metrics import precision_recall_curve
from torchmetrics.classification import PrecisionRecallCurve
import numpy as np

# folder path containing txt files
folder_path = "/home/pptr/Desktop/roboflow/yolov5/runs/predict-seg/exp/labels/"
folder_path_test = "/home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/labels/"
mAP, p, r = 0, 0, 0
how_many = 0


for filename in os.listdir(folder_path):
    how_many += 1
    preds_ann = {'boxes':[], 'scores':[], 'labels':[]}
    with open(os.path.join(folder_path, filename), "r") as file:
        for line in file:
            values = [float(value) for value in line.strip().split()]
            bbox = convert_to_bounding_box(values[1:-1])
            class_obj = int(values[0])
            confidence = values[-1]
            preds_ann['boxes'].append(bbox)
            preds_ann['scores'].append(confidence)
            preds_ann['labels'].append(class_obj)
    preds = [dict(boxes=preds_ann['boxes'], scores=preds_ann['scores'], labels=preds_ann['labels'])]
    
    target_ann = {'boxes':[], 'labels':[]}
    with open(os.path.join(folder_path_test, filename), "r") as file:
            for line in file:
                values = [float(value) for value in line.strip().split()]
                class_obj = int(values[0])
                target_ann['boxes'].append(values[1:])
                target_ann['labels'].append(class_obj)    
    target = [dict(boxes=target_ann['boxes'], labels=target_ann['labels'])]
    
    # box matching
    matches = match_boxes(np.array(target[0]['boxes']), np.array(preds[0]['boxes']))
    
    # swap
    for i in range(len(matches)):
        preds[0]['boxes'][matches[i][1]] = target[0]['boxes'][matches[i][0]]
        preds[0]['labels'][matches[i][1]] = target[0]['labels'][matches[i][0]]
    
    preds[0]['boxes'] = torch.from_numpy(np.array(preds[0]['boxes']))
    preds[0]['scores'] = torch.from_numpy(np.array(preds[0]['scores']))
    preds[0]['labels'] = torch.from_numpy(np.array(preds[0]['labels']))

    target[0]['boxes'] = torch.from_numpy(np.array(target[0]['boxes']))
    target[0]['labels'] = torch.from_numpy(np.array(target[0]['labels']))
    
    # compute metrics
    metric = MeanAveragePrecision(box_format='cxcywh')
    metric.update(preds, target)
    mAP += metric.compute()['map'].item()
    r += compute_recall(preds[0]['boxes'], preds[0]['labels'], preds[0]['scores'], target[0]['boxes'], target[0]['labels'])
    p += compute_precision(preds[0]['boxes'], preds[0]['labels'], preds[0]['scores'], target[0]['boxes'], target[0]['labels'])
                       
print(p/how_many, r/how_many, mAP/how_many)

0.9862023214076574 0.9643159716736637 0.9829967407619252


In [ ]:
## this prints bbox on an image

In [ ]:
import cv2

# read the image file
image = cv2.imread("/home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/valid/images/0006_JPG_jpg.rf.7d792aa7977eee936162616cd3dc6622.jpg")

xcenter, ycenter, width, height =  0.9484, 0.5312, 0.0969, 0.2375
# calculate the bounding box pixel coordinates
h, w, _ = image.shape
xmin = int((xcenter - width/2) * w)
ymin = int((ycenter - height/2) * h)
xmax = int((xcenter + width/2) * w)
ymax = int((ycenter + height/2) * h)

# draw the bounding box on the image
cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

# display the image with the bounding box
cv2.imshow("image", image)
cv2.waitKey(0)


QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread (0x85a8180).
Cannot move to target thread (0x83c3a40)

QObject::moveToThread: Current thread (0x83c3a40) is not the object's thread

In [ ]:
# lancio seg model e produco polygons coordinates + immagini

In [117]:
!python yolov5/segment/predict.py --save-txt --weights seg_model.pt --img 320  --source 'Step2-v1_2-mud-illu-6-beixinyao-2/test/images/*.jpg'

predict: weights=['seg_model.pt'], source=Step2-v1_2-mud-illu-6-beixinyao-2/test/images/*.jpg, data=yolov5/data/coco128.yaml, imgsz=[320, 320], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1, retina_masks=False
requirements: /home/pptr/Desktop/roboflow/yolov5/segment/requirements.txt not found, check failed.
YOLOv5 🚀 2023-2-16 Python-3.9.13 torch-1.13.1+cu117 CPU

Fusing layers... 
Model summary: 165 layers, 7401119 parameters, 0 gradients, 25.7 GFLOPs
image 1/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0006_JPG_jpg.rf.a4a0254a88a5d34201a8628a1ff1ab93.jpg: 320x320 11 coals, 4 rocks, 90.8ms
image 2/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud

image 44/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0492_JPG_jpg.rf.c29eb28540f88365246df9185693d2bd.jpg: 320x320 19 rocks, 220.4ms
image 45/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0504_JPG_jpg.rf.5600f543d9bec8ae1efc589956828fc7.jpg: 320x320 21 rocks, 288.5ms
image 46/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0510_JPG_jpg.rf.c641e409fa58f9b3e14290875b58a674.jpg: 320x320 19 rocks, 154.6ms
image 47/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0534_JPG_jpg.rf.5d84c9a5a3675e5ec21678624f95f3ef.jpg: 320x320 20 rocks, 101.3ms
image 48/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0534_JPG_jpg.rf.99512a8a2b4e29d48b746690a1e4e1e5.jpg: 320x320 20 rocks, 126.7ms
image 49/51 /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/test/images/0540_JPG_jpg.rf.1540a47f2a7b47f0ac2cc32677021717.jpg: 320x320 15 coals, 5 rocks, 84.